<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/DS_MEDICINE_DEMO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import json
from typing import Dict, Any
from openai import OpenAI
from google.colab import userdata

# =================================================================
# 1. CLIENT SETUP
# =================================================================
# NOTE: This section must be executed successfully in a Colab environment.
# api_key = userdata.get("DEEPSEEK_API_KEY")
# client = OpenAI(api_key=api_key, base_url="https://api.deepseek.com")

# Assuming 'client' is initialized before running the functions
# =================================================================
# 2. HELPER FUNCTIONS (LLM Communication and Error Handling)
# =================================================================

def _extract_and_load_json(raw_content: str) -> Dict[str, Any]:
    """Tries to extract and load a clean JSON object from LLM output, handling markdown fences."""
    raw_content = raw_content.strip()

    # 1. Clean up markdown fences (```json and ```)
    if raw_content.startswith("```"):
        raw_content = raw_content.strip("`").strip().lstrip("json").strip()

    # 2. Extract content starting from the first { and ending at the last }
    start_index = raw_content.find('{')
    end_index = raw_content.rfind('}')

    if start_index != -1 and end_index != -1 and end_index > start_index:
        cleaned_content = raw_content[start_index:end_index + 1]
        return json.loads(cleaned_content)
    else:
        # If no valid JSON structure is found, raise the error
        raise json.JSONDecodeError("Could not find or extract a valid JSON object.", raw_content, 0)

def call_deepseek_reasoner(system_role: str, user_prompt: str) -> Dict[str, Any]:
    """Handles the API call to deepseek-reasoner with robust JSON extraction."""
    print(f"\n--- Calling DeepSeek-Reasoner for: {system_role.split('Your task is to ')[1].split('.')[0]} ---")

    response = client.chat.completions.create(
        model="deepseek-reasoner",
        messages=[
            {"role": "system", "content": system_role},
            {"role": "user", "content": user_prompt},
        ],
        stream=False
    )

    raw_content = response.choices[0].message.content

    try:
        return _extract_and_load_json(raw_content)
    except json.JSONDecodeError as e:
        print(f"  !! Fatal JSONDecodeError: {e}")
        print(f"  !! Raw content start: {raw_content[:150]}...")
        raise

def call_deepseek_chat(system_role: str, user_prompt: str) -> Dict[str, Any]:
    """Handles the API call to deepseek-chat (used for coordination)."""
    print(f"\n--- Calling DeepSeek-Chat for: {system_role.split('Your task is to ')[1].split('.')[0]} ---")

    response = client.chat.completions.create(
        model="deepseek-chat",
        messages=[
            {"role": "system", "content": system_role},
            {"role": "user", "content": user_prompt},
        ],
        stream=False
    )
    return json.loads(response.choices[0].message.content)

# =================================================================
# 3. AGENT DEFINITIONS (LLM Components with Enhanced Prompting)
# =================================================================

def triage_agent() -> Dict[str, Any]:
    """Agent 1: Uses DeepSeek-Reasoner to apply USPSTF eligibility logic."""
    system_role = (
        "You are the Primary Care Triage Agent. Your task is to apply the 2021 USPSTF lung cancer screening guidelines (Age 50-80, >=20 pack-year history, and are current smokers or quit within the last 15 years) to identify the highest-priority unscreened patient. Your response MUST be a single, valid JSON object and NOTHING ELSE."
    )
    user_prompt = """
Simulate querying the EHR/Patient Registry for an eligible, unscreened patient. Create a realistic patient profile meeting all criteria.
Your output JSON MUST adhere to this structure:
{
    "patient_id": 1001,
    "name": "Jane Doe",
    "age": 62,
    "smoking_history": "30 pack-years; quit 8 years ago",
    "eligibility_check": "Patient meets 2021 USPSTF criteria for LDCT.",
    "action": "Schedule LDCT Scan"
}
"""
    return call_deepseek_reasoner(system_role, user_prompt)

def imaging_analysis_agent(patient_data: Dict[str, Any]) -> Dict[str, Any]:
    """Agent 2: Simulated specialized model for image processing."""
    print("\n--- 2. IMAGING ANALYSIS AGENT (Specialized Model) ---")
    # Simulate the output from an image analysis model (not an LLM call)
    imaging_result = {
        "scan_status": "Complete, Nodule Detected",
        "nodule_details": {
            "size_mm": 6,
            "location": "LUL (Left Upper Lobe)",
            "type": "Solid, Non-calcified",
        }
    }
    print(f"Result: {imaging_result['nodule_details']['size_mm']}mm {imaging_result['nodule_details']['type']} Nodule found.")
    return imaging_result

def risk_stratification_agent(patient_data: Dict[str, Any], imaging_data: Dict[str, Any]) -> Dict[str, Any]:
    """Agent 3: Uses DeepSeek-Reasoner to synthesize data and apply Lung-RADS criteria."""

    system_role = (
        "You are the Risk Stratification Agent. Your task is to synthesize clinical and imaging data, apply the Lung-RADS criteria, and provide a logical Chain-of-Thought for the clinical recommendation. Your response MUST be a single, valid JSON object and NOTHING ELSE."
    )

    # Safely retrieve patient info using .get()
    patient_age = patient_data.get('age', '60')
    patient_smoking = patient_data.get('smoking_history', '30 pack-years')

    user_prompt = (
        "Synthesize the following data to determine the Lung-RADS score and next clinical step:\n"
        f"Patient Info: Age: {patient_age}, Smoking History: {patient_smoking}.\n"
        f"Nodule Info: Size: {imaging_data['nodule_details']['size_mm']}mm, Type: {imaging_data['nodule_details']['type']}.\n"
        "Your final JSON MUST adhere to the following structure. Provide the Chain-of-Thought in the 'reasoning' field:\n"
        "{\n"
        '    "lung_rads_score": 3,\n'
        '    "malignancy_probability": "1-2%",\n'
        '    "reasoning": "6mm solid nodule in a high-risk patient requires a 6-month follow-up per Lung-RADS 3 guidelines.",\n'
        '    "next_step": "LDCT in 6 months"\n'
        "}"
    )

    return call_deepseek_reasoner(system_role, user_prompt)

def care_coordinator_agent(risk_data: Dict[str, Any]) -> Dict[str, Any]:
    """Agent 4: Uses DeepSeek-Chat for communication and follow-up management."""

    # Use .get() to safely retrieve the values, preventing KeyError
    lung_rads = risk_data.get('lung_rads_score', 'N/A')
    next_step = risk_data.get('next_step', 'Unknown Follow-up Protocol')

    system_role = (
        "You are the Care Coordinator Agent. Your task is to translate the clinical risk score into executable EHR updates and function calls for patient outreach. Your response MUST be a single, valid JSON object and NOTHING ELSE."
    )

    user_prompt = (
        f"Received Lung-RADS Score: {lung_rads}; Next Step: {next_step}. "
        "Execute the standard protocol: 1. Update the EHR status. 2. Initiate patient outreach function for the required follow-up appointment."
        "Your output JSON MUST adhere to this structure:\n"
        "{\n"
        '    "ehr_update_status": "Successful, status updated to 6-month follow-up",\n'
        '    "outreach_action": "Function Call: initiate_patient_outreach()"\n'
        "}"
    )

    return call_deepseek_chat(system_role, user_prompt)

# =================================================================
# 4. MAIN SIMULATION WORKFLOW
# =================================================================

def run_lcs_demo_workflow():
    print("===============================================================")
    print("🚀 STARTING MULTI-AGENT LCS DEMO (LIVE API EXECUTION STRUCTURE)")
    print("===============================================================")

    # 1. Triage Agent: Uses DeepSeek-Reasoner to find the eligible patient
    patient_triage = triage_agent()
    print(f"\n✅ **STEP 1: TRIAGE COMPLETE**")
    print(f"Patient identified: {patient_triage.get('name', 'N/A')} (ID: {patient_triage.get('patient_id', 'N/A')}).")
    print("-" * 50)

    # 2. Imaging Analysis Agent: Simulates the LDCT scan and analysis
    imaging_result = imaging_analysis_agent(patient_triage)

    # 3. Risk Stratification Agent: Uses DeepSeek-Reasoner for Lung-RADS inference
    risk_score = risk_stratification_agent(patient_triage, imaging_result)

    print(f"\n✅ **STEP 3: RISK STRATIFICATION COMPLETE**")
    print(f"Assigned Lung-RADS Score: **{risk_score.get('lung_rads_score', 'N/A')}**")
    print(f"Next Clinical Step: {risk_score.get('next_step', 'N/A')}")
    print("-" * 50)

    # 4. Care Coordinator Agent: Uses DeepSeek-Chat for follow-up execution
    coordination_result = care_coordinator_agent(risk_score)

    print("\n✅ **STEP 4: CARE COORDINATION COMPLETE**")
    print(f"EHR Status: {coordination_result.get('ehr_update_status', 'N/A')}")
    print(f"Outreach Action: {coordination_result.get('outreach_action', 'N/A')}")

    print("\n===============================================================")
    print("🎉 DEMO WORKFLOW COMPLETE. Ready for next patient.")
    print("===============================================================")

# =================================================================
# 5. EXECUTION
# =================================================================
if __name__ == "__main__":
    run_lcs_demo_workflow()

🚀 STARTING MULTI-AGENT LCS DEMO (LIVE API EXECUTION STRUCTURE)

--- Calling DeepSeek-Reasoner for: apply the 2021 USPSTF lung cancer screening guidelines (Age 50-80, >=20 pack-year history, and are current smokers or quit within the last 15 years) to identify the highest-priority unscreened patient ---

✅ **STEP 1: TRIAGE COMPLETE**
Patient identified: Robert Johnson (ID: 1002).
--------------------------------------------------

--- 2. IMAGING ANALYSIS AGENT (Specialized Model) ---
Result: 6mm Solid, Non-calcified Nodule found.

--- Calling DeepSeek-Reasoner for: synthesize clinical and imaging data, apply the Lung-RADS criteria, and provide a logical Chain-of-Thought for the clinical recommendation ---

✅ **STEP 3: RISK STRATIFICATION COMPLETE**
Assigned Lung-RADS Score: **3**
Next Clinical Step: LDCT in 6 months
--------------------------------------------------

--- Calling DeepSeek-Chat for: translate the clinical risk score into executable EHR updates and function calls for patie